In [1]:
import sys
import os
import pandas as pd
import numpy as np
import json
import math
import re
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',1000)
import sklearn
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split 
from collections import Counter
from pandas.api.types import is_dict_like
import itertools
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
sys.path.append('/Users/yantingting/PycharmProjects/modeling')
import plot_tools as pl
import metrics
import database_conncet as data_con
import summary_statistics as ss
import feature_selection as fs
from data_io_utils import *
import data_processing as dp
from jinja2 import Template

In [2]:
file_path = '/Users/yantingting/Seafile/风控/模型/14 巴西/Portocred/data/'

In [3]:
df_sdk= load_data_from_pickle(file_path,'SDK数据.pkl')
df_sdk.head(1)
flag_df = pd.read_csv(os.path.join(file_path,'flag_df.csv'))[['loan_id','sample_flag']]
flag_df['loan_id'] = flag_df['loan_id'].astype(str)
df_sdk = df_sdk.merge(flag_df,on = 'loan_id',how = 'left')
df_sdk.shape
df_sdk = df_sdk[~df_sdk['id'].isna()]
df_sdk.shape
train_sdk = df_sdk[df_sdk['sample_flag']==0]
test_sdk = df_sdk[df_sdk['sample_flag']==1]
train_sdk.shape
test_sdk.shape

,loanid,date,flag,id,app_id,system_root_status,contact_book,app_list,gps,simulator,device_info,imei,advertisement_id,current_version,uuid,app_size,phone_charging_status,electric_quantity,system_time,available_memory_size,total_memory_size,available_capacity_size,disk_size,vpn,net_type,proxy,uid,loan_id,transaction_id,request_id,package_name,create_time,message,dt,sdk_rank
0,3808097513,2019-09-19,1,5165,f4f97eeb5249228cc9b7b934047a5dff,false,"[{""name"":""caronaa"",""phones"":["" 34991600214""]},{""name"":""B"",""phones"":[""(11) 999929161""]},{""name"":""Documentaçao"",""phones"":[""(14) 991112437""]},{""name"":""Dona Fátima Gyn"",""phones"":[""(62) 84191673""]},{""name"":""Tomaz Gyn"",""phones"":[""(62) 91548002""]},{""name"":""Wigney (Márcia)"",""phones"":[""(64) 92533391""]},{""name"":""Silvino"",""phones"":[""(98) 987221685""]},{""name"":""Caixa Postal"",""phones"":[""*100""]},{""name"":""TIM"",""phones"":[""*144""]},{""name"":""MEU PLANO"",""phones"":[""*222""]},{""name"":""Flavio peru"",""phones"":[""+51 917 214 207""]},{""name"":""Alfa Emprestimo"",""phones"":[""+55 11 33868658""]},{""name"":""Juliano Jackson"",""phones"":[""+55 11 943184178""]},{""name"":""Márcio Emprestimo"",""phones"":[""+55 11 974515295""]},{""name"":""Magi créditos"",""phones"":[""+55 11 976609720""]},{""name"":""William Bulgarelli"",""phones"":[""+55 11 987596915""]},{""name"":""Anderson Faciroli Divulgador"",""phones"":[""+55 16 992266793""]},{""name"":""Bruno Divulgador"",""phones"":[""+55 16 992322280""]},{""name"":""Fernando Talk Radio"",""phones"":[""+55 18 988187076""]},{""name"":""Edi...","[{""appName"":""Grana Fácil"",""installTime"":""2019-09-16 08:54:51:051"",""packageName"":""com.fintopia.brazilcash"",""versionCode"":20005,""versionName"":""2.0.5""},{""appName"":""Serasa"",""installTime"":""2019-06-20 03:30:40:040"",""packageName"":""br.com.serasaexperian.consumidor"",""versionCode"":523,""versionName"":""6.2.2""},{""appName"":""Shazam"",""installTime"":""2018-08-18 08:42:55:055"",""packageName"":""com.shazam.android"",""versionCode"":904800,""versionName"":""9.48.0-190906""},{""appName"":""Connectmix Ranking Musical"",""installTime"":""2019-03-12 00:13:40:040"",""packageName"":""com.connectmix.ranking.musical"",""versionCode"":10205,""versionName"":""1.2.5""},{""appName"":""WhatsApp"",""installTime"":""2017-06-06 17:27:04:004"",""packageName"":""com.whatsapp"",""versionCode"":452954,""versionName"":""2.19.244""},{""appName"":""Previsão de Tempo"",""installTime"":""2019-09-14 15:57:58:058"",""packageName"":""br.inpe.cptec.www.previsao_do_tempo"",""versionCode"":2,""versionName"":""1.0.1""},{""appName"":""Caixa"",""installTime"":""2018-08-13 10:06:17:017"",""packageName"":""br.com...","[-18.156123333333333,-47.92740333333333]",false,"{""manufacture"":""LGE"",""google_version"":""NRD90U"",""version_release"":""7.0"",""heightPixels"":""1199"",""model"":""LG-M400"",""sdk_int"":""24"",""brand"":""lge"",""device"":""ANDROID"",""board"":""mt6755"",""widthPixels"":""720""}",356950080223718,None,2.4.10,00000000-3933-78bc-0000-00000036f3bb,238,0,82,2019-09-19 07:32:27:027,424 MB,"1,83 GB","1,03 GB","23,62 GB",false,1,false,,3808097513,Ae5fe1ca43a6dacca2c2aa52a3d09aa0ca78ce61d-1568889155069,Td54be3e7d37f5373f9486b1a0f42f23f6d44325c-1568889155070,br.com.inueva.contablet.portocred,2019-09-19 10:32:36.0,None,20191016,1.0


(445, 36)

(435, 36)

(323, 36)

(112, 36)

# APP

In [11]:
df_app = df_sdk[['loan_id','app_list']]
df_app = dp.from_json(df_app, 'app_list')
df_app.to_csv(file_path + 'app_data_analysis.csv')

app_data1 = train_sdk[['loan_id','app_list']]
app_data1 = dp.from_json(app_data1, 'app_list')
app_data1.to_csv(file_path + 'df_app_train1.csv')

new columns done
versionName
versionCode
packageName
appName
installTime
run time = 0.0696871280670166
new columns done
versionName
versionCode
packageName
appName
installTime
run time = 0.08658885955810547


## 频数 

In [13]:
high_cut = 0.4 
mid_cut = 0.2
tfidf_cut = 0.1
app_data1 = pd.read_csv(file_path + 'df_app_train1.csv')
app_freq = pd.DataFrame(app_data1['appName'].value_counts()).reset_index()
print('训练集的不同app数量: ' , app_freq.shape[0])
app_freq.rename(columns={'index': 'app', 'appName': 'freq'}, inplace=True)
cnt_loan = app_data1['loan_id'].nunique()
app_freq['rate_freq'] = app_freq['freq'].apply(lambda x: x/cnt_loan)
app_tfidf = app_freq[app_freq['rate_freq']>=float('%f'%tfidf_cut)]
print('可用于tf_idf的app总数是(安装率超过 %f)： '%tfidf_cut, app_tfidf.shape[0])
app_tfidf.to_csv(file_path + 'app_for_tfidf.csv', index = False)
app_freq['high_freq_app'] = app_freq['rate_freq'].apply(lambda x: 1 if x >= float('%f'%high_cut) else 0)
print('高频APP个数', app_freq['high_freq_app'].sum())
app_freq['mid_freq_app'] = app_freq['rate_freq'].apply(lambda x: 1 if (float('%f'%mid_cut)<= x <float('%f'%high_cut)) else 0)
print('中频APP个数', app_freq['mid_freq_app'].sum())
app_freq['low_freq_app'] = app_freq['freq'].apply(lambda x: 1 if x <= 1 else 0)
print('低频APP个数(仅一人安装)', app_freq['low_freq_app'].sum())
app_freq.to_csv(file_path  + 'app_freq_train_dict.csv', index=False)
time.sleep(0.1)

训练集的不同app数量:  3656
可用于tf_idf的app总数是(安装率超过 0.100000)：  85
高频APP个数 10
中频APP个数 32
低频APP个数(仅一人安装) 2499


In [14]:
time.sleep(0.1)
df_app = pd.read_csv(file_path + 'app_data_analysis.csv')
count_app = Counter(df_app['loan_id'])
app_freq = pd.read_csv(file_path  + 'app_freq_train_dict.csv')
train1 = pd.merge(df_app, app_freq, left_on='appName', right_on='app', how = 'left')
var_app = train1.groupby(['loan_id'])['high_freq_app', 'mid_freq_app', 'low_freq_app'].agg('sum').reset_index()
var_app['cnt_app'] = var_app['loan_id'].map(lambda x: count_app[x])
list1 = ['high_freq_app', 'mid_freq_app', 'low_freq_app']
for i in list1:
    var_app['rate_' + i] = var_app.apply(lambda x: x[i]/x['cnt_app'], axis=1)
var_app.to_csv(file_path + 'var_app_freq.csv', index=False)
var_dict_app_freq = dp.VarDict(var_app.drop('loan_id', axis = 1), data_sorce='app_freq')
var_dict_app_freq.to_csv(os.path.join(file_path, 'var_dict_app_freq.csv'), index = False)
print('app频数变量: ', var_app.shape)
var_app.head()

app频数变量:  (435, 8)


/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,loan_id,high_freq_app,mid_freq_app,low_freq_app,cnt_app,rate_high_freq_app,rate_mid_freq_app,rate_low_freq_app
0,3808097513,8.0,9.0,2.0,42,0.190476,0.214286,0.047619
1,3808125361,3.0,3.0,5.0,22,0.136364,0.136364,0.227273
2,3808150997,4.0,14.0,5.0,37,0.108108,0.378378,0.135135
3,3808162998,7.0,9.0,19.0,93,0.075269,0.096774,0.204301
4,3808170569,5.0,15.0,6.0,55,0.090909,0.272727,0.109091


## （TF-IDF）

In [15]:
time.sleep(0.5)
df_app_train1 = pd.read_csv(file_path + 'df_app_train1.csv')
app_idf_data = df_app_train1[['loan_id','appName']].copy()
app_idf = pd.pivot_table(app_idf_data.drop_duplicates(), index = 'appName', values = 'loan_id', aggfunc = 'count').reset_index()
app_idf.rename(columns={'loan_id': 'cnt_idf'}, inplace=True)
cnt_document = app_idf_data['loan_id'].nunique()
app_idf['idf'] = app_idf['cnt_idf'].apply(lambda x: math.log10(cnt_document/(x+1)))
app_idf[['appName', 'idf']].to_csv(file_path + 'app_idf_train1_dict.csv', index=False)
app_idf.shape
app_idf.head()

(3656, 3)

,appName,cnt_idf,idf
0,Abastece Aí,15,1.305083
1,#Associe-se,1,2.208173
2,%,1,2.208173
3,.Escola RS - Gestor,1,2.208173
4,12 MESES,1,2.208173


In [16]:
app_idf.head(100)

,appName,cnt_idf,idf
0,Abastece Aí,15,1.305083
1,#Associe-se,1,2.208173
2,%,1,2.208173
3,.Escola RS - Gestor,1,2.208173
4,12 MESES,1,2.208173
5,155 Palavras cruzadas,1,2.208173
6,17TRACK,1,2.208173
7,1945 Air Force,1,2.208173
8,1xBet,1,2.208173
9,21 Buttons,1,2.208173


In [33]:
time.sleep(0.5)
df_app = pd.read_csv(file_path + 'app_data_analysis.csv')
print(df_app.shape)
dict_tfidf = pd.read_csv(file_path + 'app_idf_train1_dict.csv')
app_tfidf_data = df_app.copy()
# 计算每个loan的app个数
cnt_app = Counter(app_tfidf_data['loan_id'])
# 计算TF
app_tfidf =  pd.pivot_table(app_tfidf_data, index=['loan_id', 'appName'], values='packageName',aggfunc='count' ,fill_value=0).reset_index()
app_tfidf.rename(columns = {'packageName': 'cnt_tf'},  inplace=True)
app_tfidf['cnt_app'] = app_tfidf['loan_id'].map(lambda x: cnt_app[x])
app_tfidf['app_tf'] = app_tfidf.apply(lambda x: x['cnt_tf']/x['cnt_app'], axis = 1)

# 计算IDF
app_tfidf = pd.merge(app_tfidf, dict_tfidf, on='appName', how = 'left')
app_tfidf['tf_idf'] = app_tfidf.apply(lambda x: x['app_tf']*x['idf'], axis=1)

app_for_tfidf = pd.read_csv(file_path + 'app_for_tfidf.csv')
app_tfidf_list = app_for_tfidf['app'].values.tolist()
app_tfidf1=app_tfidf[app_tfidf['appName'].isin(app_tfidf_list)]
var_app_tfidf = pd.pivot_table(app_tfidf1, index='loan_id', columns = 'appName',values='tf_idf', aggfunc='sum', fill_value=0).reset_index()
var_app_tfidf = pd.merge(df_app[['loan_id']].drop_duplicates(),var_app_tfidf, on='loan_id' , how='left')
print(var_app_tfidf.shape)
var_app_tfidf.to_csv(file_path + 'var_app_tfidf.csv', index = False)
var_dict_app_tfidf = dp.VarDict(var_app_tfidf.drop('loan_id', axis = 1), data_sorce='app_tfidf')
var_dict_app_tfidf.to_csv(os.path.join(file_path, 'var_dict_app_tfidf.csv'), index = False)


(20567, 7)
(435, 84)


# CONTACT

In [9]:
contact_data = df_sdk[['loan_id','contact_book']]
# contact数据  1 ）删除contact为空的数据
print('********** contact 原始样本量: **********')
contact_data.shape
contact_data = contact_data[contact_data['contact_book'].notnull()]
contact_data = contact_data.drop_duplicates()
print('********** contact 去重之后样本量: **********')
contact_data.shape
contact_data1 = dp.from_json(contact_data, 'contact_book')
print('********** contact 解析之后数量: **********')
contact_data1.info()
contact_data1.to_csv(file_path + 'contact_data_analysis.csv', index = False)

********** contact 原始样本量: **********


(435, 2)

********** contact 去重之后样本量: **********


(435, 2)

new columns done
phones
name
run time = 0.5906727313995361
********** contact 解析之后数量: **********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140262 entries, 0 to 140261
Data columns (total 3 columns):
loan_id    140262 non-null object
phones     140262 non-null object
name       140262 non-null object
dtypes: object(3)
memory usage: 3.2+ MB


In [10]:
df_contact = pd.read_csv(file_path + 'contact_data_analysis.csv')
df_contact.nunique()
close_relative = pd.read_excel(os.path.join('/Users/yantingting/Documents/海外风控/巴西/', 'close_relative.xlsx'))
close_list = list(set(close_relative['BaXi']))
df_contact['phones1'] = df_contact['phones'].apply(lambda x: re.sub('\D', '', x.replace('+55', '')) if not isinstance(x,float) else np.nan)
df_contact['len_phones'] = df_contact['phones1'].apply(lambda x: len(x) if not isinstance(x,float) else np.nan)
df_contact['len_is9'] = df_contact['len_phones'].apply(lambda x: 1 if x==9 else 0)
df_contact['len_is10'] = df_contact['len_phones'].apply(lambda x: 1 if x==10 else 0)
df_contact['len_is11'] = df_contact['len_phones'].apply(lambda x: 1 if x==11 else 0)
df_contact['len_is12'] = df_contact['len_phones'].apply(lambda x: 1 if x==12 else 0)
df_contact['len_isless6'] = df_contact['len_phones'].apply(lambda x: 1 if x<=6 else 0)
df_contact['len_isover13'] = df_contact['len_phones'].apply(lambda x: 1 if x>=13 else 0)
df_contact['is_close_relative'] = df_contact['name'].apply(lambda x: 1 if x in close_list else 0)
df_contact.head()
df_contact['loan_id'].nunique()

loan_id       435
phones     129431
name        98780
dtype: int64

,loan_id,phones,name,phones1,len_phones,len_is9,len_is10,len_is11,len_is12,len_isless6,len_isover13,is_close_relative
0,3808097513,[' 34991600214'],caronaa,34991600214,11,0,0,1,0,0,0,0
1,3808097513,['(11) 999929161'],B,11999929161,11,0,0,1,0,0,0,0
2,3808097513,['(14) 991112437'],Documentaçao,14991112437,11,0,0,1,0,0,0,0
3,3808097513,['(62) 84191673'],Dona Fátima Gyn,6284191673,10,0,1,0,0,0,0,0
4,3808097513,['(62) 91548002'],Tomaz Gyn,6291548002,10,0,1,0,0,0,0,0


435

In [11]:
var_contact_data1 = df_contact.groupby(['loan_id'])
var_contact_data = pd.DataFrame()
var_contact_data['cnt_phone'] = var_contact_data1['phones1'].count()
var_contact_data['cnt_unique_phone'] = var_contact_data1['phones1'].nunique()
var_contact_data['cnt_names'] = var_contact_data1['name'].count()
var_contact_data['cnt_unique_names'] = var_contact_data1['name'].nunique()
var_contact_data['cnt_close_relative'] = var_contact_data1['is_close_relative'].sum()
var_contact_data['cnt_len_is9'] = var_contact_data1['len_is9'].sum()
var_contact_data['cnt_len_is10'] = var_contact_data1['len_is10'].sum()
var_contact_data['cnt_len_is11'] = var_contact_data1['len_is11'].sum()
var_contact_data['cnt_len_is12'] = var_contact_data1['len_is12'].sum()
var_contact_data['cnt_len_isless6'] = var_contact_data1['len_isless6'].sum()
print(2)
var_contact_data['cnt_len_isover13'] = var_contact_data1['len_isover13'].sum()
var_contact_data['rate_close_relative'] = var_contact_data['cnt_close_relative']/var_contact_data['cnt_phone']
var_contact_data['rate_len_is9'] = var_contact_data['cnt_len_is9']/var_contact_data['cnt_phone']
var_contact_data['rate_len_is10'] = var_contact_data['cnt_len_is10']/var_contact_data['cnt_phone']
var_contact_data['rate_len_is11'] = var_contact_data['cnt_len_is11']/var_contact_data['cnt_phone']
var_contact_data['rate_len_is12'] = var_contact_data['cnt_len_is12']/var_contact_data['cnt_phone']
var_contact_data['rate_len_isless6'] = var_contact_data['cnt_len_isless6']/var_contact_data['cnt_phone']
var_contact_data['rate_len_isover13'] = var_contact_data['cnt_len_isover13']/var_contact_data['cnt_phone']
var_contact_data = var_contact_data.reset_index().drop_duplicates('loan_id', keep = 'last')
var_contact_data.head()
var_contact_data.shape
var_dict_contact = dp.VarDict(var_contact_data.drop('loan_id', axis = 1), data_sorce='contact')
var_dict_contact.to_csv(os.path.join(file_path, 'var_dict_contact.csv'), index = False)
var_contact_data.to_csv(os.path.join(file_path, 'var_contact.csv'), index = False)
print('end!')

2


,loan_id,cnt_phone,cnt_unique_phone,cnt_names,cnt_unique_names,cnt_close_relative,cnt_len_is9,cnt_len_is10,cnt_len_is11,cnt_len_is12,cnt_len_isless6,cnt_len_isover13,rate_close_relative,rate_len_is9,rate_len_is10,rate_len_is11,rate_len_is12,rate_len_isless6,rate_len_isover13
0,3808097513,496,484,496,470,0,183,218,53,6,11,9,0.0,0.368952,0.439516,0.106855,0.012097,0.022177,0.018145
1,3808125361,344,271,344,269,0,183,4,83,43,4,15,0.0,0.531977,0.011628,0.241279,0.125000,0.011628,0.043605
2,3808150997,67,67,67,67,0,18,1,28,6,10,0,0.0,0.268657,0.014925,0.417910,0.089552,0.149254,0.000000
3,3808162998,206,196,206,197,0,3,7,42,136,17,0,0.0,0.014563,0.033981,0.203883,0.660194,0.082524,0.000000
4,3808170569,121,121,121,120,0,85,2,18,3,1,1,0.0,0.702479,0.016529,0.148760,0.024793,0.008264,0.008264


(435, 19)

end!


# deviceinfo

In [5]:
deviceinfo_data = df_sdk[['loan_id','device_info']]
# keys_list = json.loads(deviceinfo_data['device_info'][0])
# for keys in keys_list:
#     deviceinfo_data[keys] = deviceinfo_data['device_info'].apply(lambda x: json.loads(x).get(keys))
# deviceinfo_data = deviceinfo_data.drop('device_info', axis = 1)
deviceinfo_data = dp.from_json(deviceinfo_data, 'device_info')
deviceinfo_data.to_csv(file_path + 'device_data_analysis.csv', index= False)
deviceinfo_data.info()

new columns done
version_release
widthPixels
google_version
sdk_int
board
device
heightPixels
brand
model
manufacture
run time = 0.022752046585083008
<class 'pandas.core.frame.DataFrame'>
Int64Index: 435 entries, 0 to 444
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   loan_id          435 non-null    object
 1   version_release  435 non-null    object
 2   widthPixels      435 non-null    object
 3   google_version   435 non-null    object
 4   sdk_int          435 non-null    object
 5   board            435 non-null    object
 6   device           435 non-null    object
 7   heightPixels     435 non-null    object
 8   brand            435 non-null    object
 9   model            435 non-null    object
 10  manufacture      435 non-null    object
dtypes: object(11)
memory usage: 60.8+ KB


In [3]:
deviceinfo_data = pd.read_csv(file_path + 'device_data_analysis.csv')
deviceinfo_data['manufacture'] = deviceinfo_data['manufacture'].astype(str).str.upper()
deviceinfo_data['board'] = deviceinfo_data['board'].astype(str).str.upper()
deviceinfo_data['widthPixels'] = deviceinfo_data['widthPixels'].astype(str)
deviceinfo_data['brand'] = deviceinfo_data['brand'].astype(str).str.upper()
deviceinfo_data['heightPixels'] = deviceinfo_data['heightPixels'].astype(str)
deviceinfo_data['model']=deviceinfo_data['model'].astype(str)
deviceinfo_data['screen'] = '(' + round(pd.to_numeric(deviceinfo_data['heightPixels']), 0).astype(str) + ', ' \
                 + round(pd.to_numeric(deviceinfo_data['widthPixels']), 0).astype(str) + ')'
deviceinfo_data.head()

,loan_id,board,model,google_version,version_release,sdk_int,device,widthPixels,manufacture,brand,heightPixels,screen
0,3808097513,MT6755,LG-M400,NRD90U,7.0,24,ANDROID,720,LGE,LGE,1199,"(1199, 720)"
1,3808125361,MSM8909,LM-X210,N2G47H,7.1.2,25,ANDROID,720,LGE,LGE,1196,"(1196, 720)"
2,3808150997,MSM8937,SM-J610G,PPR1.180610.011.J610GUBS3BSH3,9,28,ANDROID,720,SAMSUNG,SAMSUNG,1396,"(1396, 720)"
3,3808162998,MT6753,5085N,MRA58K release-keys,6.0,23,ANDROID,720,TCL,TCL,1280,"(1280, 720)"
4,3808170569,MSM8953,motorola one,PPKS29.68-16-21-11,9,28,ANDROID,720,MOTOROLA,MOTOROLA,1362,"(1362, 720)"


In [14]:
dummy_list = ['manufacture',
 'board',
 'model',
 'screen']
frame1 = deviceinfo_data[['loan_id']].copy()
for cols in dummy_list:
    temp_list = deviceinfo_data[cols].value_counts().index.tolist()
    temp_dummy = pd.get_dummies(deviceinfo_data[cols])[temp_list]
    temp_dummy.columns = [cols + '_' + str(col) for col in temp_dummy.columns]
    frame1 = frame1.join(temp_dummy)
var_deviceinfo = frame1
var_deviceinfo.to_csv(file_path +'var_deviceinfo.csv', index = False)
var_deviceinfo.shape
var_deviceinfo.head() 
var_dict_deviceinfo = dp.VarDict(var_deviceinfo.drop('loan_id', axis = 1), data_sorce='deviceinfo')
var_dict_deviceinfo.to_csv(os.path.join(file_path, 'var_dict_deviceinfo.csv'), index = False)

(435, 242)

,loan_id,manufacture_SAMSUNG,manufacture_MOTOROLA,manufacture_LGE,manufacture_ASUS,manufacture_XIAOMI,manufacture_SONY,manufacture_LENOVO,manufacture_HMD GLOBAL,manufacture_HTC,manufacture_BLU,manufacture_LG ELECTRONICS,manufacture_TCL,manufacture_UMIDIGI,board_MSM8953,board_UNIVERSAL7870,board_MSM8937,board_UNIVERSAL7885,board_MSM8916,board_MT6755,board_UNIVERSAL7570,board_MT6737T,board_SDM660,board_UNKNOWN,board_SDM845,board_UNIVERSAL8890,board_MT6735,board_UNIVERSAL7580,board_MSM8952,board_UNIVERSAL7570_GO,board_SDM636,board_UNIVERSAL9820,board_UNIVERSAL3475,board_UNIVERSAL8895,board_ERD9610,board_MSM8937_32,board_MSM8976,board_MT6765,board_UNIVERSAL9610,board_UNIVERSAL7880,board_CEPHEUS,board_MSM8998,board_ASUS_X00ID,board_YSL,board_MT6753,board_MSM8917,board_MOOREFIELD,board_SC9830I,board_A2016B30,board_BLU_R2,board_SC7730SE,board_A5_PRO,board_MSM8909,board_MSM8996,board_MT6750T,board_MT6737,board_MSM8994,model_SM-G610M,model_motorola one,model_moto g(7) play,model_SM-J500M,model_moto g(6) play,model_SM-G570M,model_SM-J600GT,model_SM-J810M,model_SM-J610G,model_SM-G532MT,model_SM-A105M,model_LG-M250,model_SM-A305GT,model_SM-J710MN,model_SM-A205G,model_SM-J400M,model_Moto G (5S) Plus,model_SM-J730G,model_LG-M320,model_Moto G (5),model_moto g(7),model_LM-X410.F,model_Moto G (5S),model_SM-J415G,model_SM-J410G,model_SM-J260M,model_moto g(6),model_SM-J700M,model_SM-J530G,model_SM-A730F,model_moto e5,model_SM-J701MT,model_LG-K430,model_SM-G9600,model_moto g(7) power,model_Moto G (5) Plus,model_SM-G935F,model_Moto G Play,model_SM-G9650,model_Moto G (4),model_ASUS_A001D,model_SM-A750G,model_SM-G532M,model_moto g(6) plus,model_SM-G611MT,model_LG-M700,model_SM-G975F,model_XT1635-02,model_LM-X420,model_ASUS_X00ID,model_ASUS_X00LD,model_Redmi Note 6 Pro,model_SM-A720F,model_SM-A910F,model_MotoG3,model_Moto Z3 Play,model_motorola one vision,model_Lenovo K10a40,model_Redmi Note 7,model_moto e5 play,model_SM-G930F,model_SM-G600FY,model_SM-A505GT,model_MI 9,model_SM-J200BT,model_SM-G950F,model_motorola one action,model_ASUS_Z00AD,model_MI PLAY,model_5085N,model_SM-A500M,model_ASUS_X018D,model_E6853,model_ASUS_X00LDA,model_SM-J510MN,model_Moto C Plus,model_Redmi S2,model_Moto E (4) Plus,model_ASUS_Z012DC,model_Nokia 6.1 Plus,model_SM-G903M,model_SM-M305M,model_F3116,model_SM-N9600,model_moto g(7) plus,model_LG-H540,model_HTC U11,model_Mi A2 Lite,model_Redmi Note 5,model_LM-X210,model_LG-M400,model_ASUS_Z017DC,model_SM-G973F,model_R2 LTE,model_SM-G955F,model_SM-J120M,model_A5_Pro,model_G3226,model_Moto E (4),model_Lenovo A2016b30,model_ASUS_X00TDB,model_LG-H630,model_SM-J320M,model_MI 6,model_Redmi 6,model_LG-X230,model_Moto G4,model_Mi A2,model_SM-G531H,model_LG-K350,model_Redmi 6A,model_ASUS_X017DA,model_ASUS_Z01KD,model_SM-A530F,model_LG-H870,"screen_(1280, 720)","screen_(1920, 1080)","screen_(1344, 720)","screen_(1776, 1080)","screen_(1384, 720)","screen_(1193, 720)","screen_(1396, 720)","screen_(960, 540)","screen_(1184, 720)","screen_(2016, 1080)","screen_(1362, 720)","screen_(2131, 1080)","screen_(2076, 1080)","screen_(2094, 1080)","screen_(1423, 720)","screen_(1382, 720)","screen_(1376, 720)","screen_(888, 540)","screen_(1371, 720)","screen_(2004, 1080)","screen_(2792, 1440)","screen_(1372, 720)","screen_(2135, 1080)","screen_(2062, 1080)","screen_(2064, 1080)","screen_(888, 480)","screen_(2130, 1080)","screen_(2136, 1080)","screen_(1351, 720)","screen_(2560, 1440)","screen_(2065, 1080)","screen_(2053, 1080)","screen_(2043, 1080)","screen_(2030, 1080)","screen_(2055, 1080)","screen_(782, 480)","screen_(1199, 720)","screen_(1358, 720)","screen_(800, 480)","screen_(2181, 1080)","screen_(2672, 1440)","screen_(2048, 1080)","screen_(854, 480)","screen_(1399, 720)","screen_(1798, 1080)","screen_(1187, 720)","screen_(1370, 720)","screen_(1196, 720)","screen_(2028, 1080)","screen_(1200, 720)","screen_(2095, 1080)","screen_(2191, 1080)","screen_(2128, 1080)","screen_(1816, 1080)","screen_(1454, 720)","screen_(2078, 1080)","screen_(2208, 1080

# othersdk

In [15]:
other_list = ['loan_id','gps', 'app_size','phone_charging_status', 'electric_quantity',
              'available_memory_size', 'total_memory_size', 'available_capacity_size',
               'disk_size', 'vpn', 'net_type']
othersdk_data = df_sdk[other_list]
othersdk_data.shape
othersdk_data.head()
var_othersdk = othersdk_data[['loan_id', 'app_size','phone_charging_status', 'electric_quantity']].copy()
size_list = ['available_memory_size', 'total_memory_size', 'available_capacity_size', 'disk_size']
for i in size_list: 
    othersdk_data[i] =  othersdk_data[i].apply(lambda x: x.replace(',', '.').replace(' ', ''))
var_othersdk['gps'] = othersdk_data['gps'].apply(lambda x: 0 if (x =='[0.0, 0.0]') | (x =='\"[0.0,0.0]\"') else 1)
var_othersdk['available_memory_size'] = othersdk_data['available_memory_size'].apply(lambda x: float(x.replace('MB', '')) if x[-2:] == 'MB' else float(x.replace('GB', ''))*1024 if x[-2:] == 'GB' else 0)
var_othersdk['total_memory_size'] = othersdk_data['total_memory_size'].apply(lambda x: float(x.replace('MB', '')) if x[-2:] == 'MB' else float(x.replace('GB', ''))*1024 if x[-2:] == 'GB' else 0)
var_othersdk['available_capacity_size'] = othersdk_data['available_capacity_size'].apply(lambda x: float(x.replace('MB', '')) if x[-2:] == 'MB' else float(x.replace('GB', ''))*1024 if x[-2:] == 'GB' else 0)
var_othersdk['disk_size'] = othersdk_data['disk_size'].apply(lambda x: float(x.replace('MB', '')) if x[-2:] == 'MB' else float(x.replace('GB', ''))*1024 if x[-2:] == 'GB' else 0)
var_othersdk['vpn'] = othersdk_data['vpn'].apply(lambda x : 1 if x == 'True' else 0)
var_othersdk['net_type']  = othersdk_data['net_type']
var_othersdk.to_csv(file_path + 'var_othersdk.csv', index=False)
var_othersdk.shape
var_othersdk.head() 
var_dict_othersdk = dp.VarDict(var_othersdk.drop('loan_id', axis = 1), data_sorce='othersdk')
var_dict_othersdk.to_csv(os.path.join(file_path, 'var_dict_othersdk.csv'), index = False)

(435, 11)

,loan_id,gps,app_size,phone_charging_status,electric_quantity,available_memory_size,total_memory_size,available_capacity_size,disk_size,vpn,net_type
0,3808097513,"[-18.156123333333333,-47.92740333333333]",238,0,82,424 MB,"1,83 GB","1,03 GB","23,62 GB",false,1
1,3808125361,"[-23.71034458,-46.87601053]",216,2,43,653 MB,"1,86 GB","1,63 GB","9,50 GB",false,3
2,3808150997,"[-23.55134579,-46.82475019]",336,2,69,842 MB,"2,96 GB","15,34 GB","25,82 GB",false,1
3,3808162998,"[0.0, 0.0]",337,2,0,"1,00 GB","2,82 GB","6,36 GB","14,59 GB",false,1
4,3808170569,"[0.0, 0.0]",238,0,27,"0,99 GB","3,75 GB","34,06 GB","55,10 GB",false,1


/Users/yantingting/anaconda3/envs/modeling/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


(435, 11)

,loan_id,app_size,phone_charging_status,electric_quantity,gps,available_memory_size,total_memory_size,available_capacity_size,disk_size,vpn,net_type
0,3808097513,238,0,82,1,424.00,1873.92,1054.72,24186.88,0,1
1,3808125361,216,2,43,1,653.00,1904.64,1669.12,9728.00,0,3
2,3808150997,336,2,69,1,842.00,3031.04,15708.16,26439.68,0,1
3,3808162998,337,2,0,0,1024.00,2887.68,6512.64,14940.16,0,1
4,3808170569,238,0,27,0,1013.76,3840.00,34877.44,56422.40,0,1


In [16]:
print(1)

1
